In [35]:
import numpy as np
import pandas as pd
import os
from pathlib import Path

import matplotlib.pyplot as plt
%matplotlib inline
from tqdm import tqdm_notebook as tqdm
from sklearn.preprocessing import StandardScaler
from sklearn.svm import NuSVR, SVR
from sklearn.metrics import mean_absolute_error
pd.options.display.precision = 15

from joblib import Parallel, delayed

import lightgbm as lgb
import xgboost as xgb
import time
import datetime
from catboost import CatBoostRegressor
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import StratifiedKFold, KFold, RepeatedKFold
from sklearn.metrics import mean_absolute_error
from sklearn.linear_model import LinearRegression

from scipy import linalg
#from scipy import signal
from scipy.signal import sosfilt
from scipy.signal import butter
import pywt
from tsfresh.feature_extraction import feature_calculators

import gc
import seaborn as sns
import warnings
warnings.filterwarnings("ignore")

from scipy.signal import hilbert
from scipy.signal import hann
from scipy.signal import convolve
from scipy import stats
from sklearn.kernel_ridge import KernelRidge

In [2]:
import chainer.functions as F

In [3]:
DataPath = Path('../data/train_wave_split')
delimiter = ','

rows = 150_000
n_samples = 150_000
sample_duration = 0.02
sample_rate = n_samples * (1 / sample_duration)

In [4]:
def maddest(d, axis=None):
    return np.mean(np.absolute(d - np.mean(d, axis)), axis)

def high_pass_filter(x, low_cutoff=1000, sample_rate=sample_rate):
    nyquist = 0.5 * sample_rate
    norm_low_cutoff = low_cutoff / nyquist
    sos = butter(10, Wn=[norm_low_cutoff], btype='highpass', output='sos')
    filtered_sig = sosfilt(sos, x)

    return filtered_sig

def denoise_signal(x, wavelet='db4', level=1):
    coeff = pywt.wavedec( x, wavelet, mode="per" )
    sigma = (1/0.6745) * maddest( coeff[-level] )
    uthresh = sigma * np.sqrt( 2*np.log( len( x ) ) )
    coeff[1:] = ( pywt.threshold( i, value=uthresh, mode='hard' ) for i in coeff[1:] )

    return pywt.waverec( coeff, wavelet, mode='per' )

In [5]:
def add_trend_feature(arr, abs_values=False):
    idx = np.array(range(len(arr)))
    lr = LinearRegression()
    lr.fit(idx.reshape(-1, 1), arr)
    return lr.coef_[0]

def classic_sta_lta(x, length_sta, length_lta):
    
    sta = np.cumsum(x ** 2)

    # Convert to float
    sta = np.require(sta, dtype=np.float)

    # Copy for LTA
    lta = sta.copy()

    # Compute the STA and the LTA
    sta[length_sta:] = sta[length_sta:] - sta[:-length_sta]
    sta /= length_sta
    lta[length_lta:] = lta[length_lta:] - lta[:-length_lta]
    lta /= length_lta

    # Pad zeros
    sta[:length_lta - 1] = 0

    # Avoid division by zero by setting zero values to tiny float
    dtiny = np.finfo(0.0).tiny
    idx = lta < dtiny
    lta[idx] = dtiny

    return sta / lta

def calc_change_rate(x):
    change = (np.diff(x) / x[:-1]).values
    change = change[np.nonzero(change)[0]]
    change = change[~np.isnan(change)]
    change = change[change != -np.inf]
    change = change[change != np.inf]
    return np.mean(change)

In [6]:
def feature_engineering(x):
    x =  pd.Series(x)
    x_abs = np.abs(x)
    x_max = x.max()
    x_min = x.min()
    
    X_features = pd.Series(dtype=np.float64)
    
    X_features.loc['mean'] = x.mean()
    X_features.loc['std'] = x.std()
    X_features.loc['max'] = x_max
    X_features.loc['min'] = x_min
    X_features.loc['mad'] = x.mad()
    X_features.loc['kurt'] = x.kurtosis()
    X_features.loc['skew'] = x.skew()
    #X_features.loc['med'] = x.median()
    
    X_features.loc['max_to_min'] = x_max / np.abs(x_min)
    X_features.loc['max_to_min_diff'] = x_max - np.abs(x_min)
    X_features.loc['max_min_diff'] = x_max - x_min
    X_features.loc['count_big'] = len(x[x_abs > 1000])
    
    X_features.loc['mean_change_abs'] = np.mean(np.diff(x))
    X_features.loc['mean_change_rate'] = calc_change_rate(x)
    X_features.loc['abs_max'] = x_abs.max()
    #X_features.loc['abs_min'] = x_abs.min()
    X_features.loc['abs_mean'] = x_abs.mean()
    X_features.loc['abs_std'] = x_abs.std()


    X_features.loc['mean_change_rate_first_50000'] = calc_change_rate(x[:50000])
    X_features.loc['mean_change_rate_last_50000'] = calc_change_rate(x[-50000:])
    X_features.loc['mean_change_rate_first_10000'] = calc_change_rate(x[:10000])
    X_features.loc['mean_change_rate_last_10000'] = calc_change_rate(x[-10000:])
    
    X_features.loc['q999'] = np.quantile(x,0.999)
    X_features.loc['q99'] = np.quantile(x, 0.99)
    X_features.loc['q95'] = np.quantile(x, 0.95)
    X_features.loc['q05'] = np.quantile(x, 0.05)
    X_features.loc['q01'] = np.quantile(x, 0.01)
    X_features.loc['q001'] = np.quantile(x,0.001)
    
#     X_features.loc['abs_q95'] = np.quantile(x_abs, 0.95)
#     X_features.loc['abs_q99'] = np.quantile(x_abs, 0.99)
#     X_features.loc['abs_q05'] = np.quantile(x_abs, 0.05)
#     X_features.loc['abs_q01'] = np.quantile(x_abs, 0.01)
    
    X_features.loc['trend'] = add_trend_feature(x)
    X_features.loc['abs_trend'] = add_trend_feature(x_abs)
        
#     X_features.loc['Hilbert_mean'] = np.abs(hilbert(x)).mean()
    X_features.loc['Hann_window_mean'] = (convolve(x, hann(150), mode='same') / sum(hann(150))).mean()
    X_features.loc['classic_sta_lta1_mean'] = classic_sta_lta(x, 500, 10000).mean()
    X_features.loc['classic_sta_lta2_mean'] = classic_sta_lta(x, 5000, 100000).mean()
    X_features.loc['classic_sta_lta3_mean'] = classic_sta_lta(x, 3333, 6666).mean()
    X_features.loc['classic_sta_lta4_mean'] = classic_sta_lta(x, 10000, 25000).mean()
    X_features.loc['classic_sta_lta5_mean'] = classic_sta_lta(x, 50, 1000).mean()
    X_features.loc['classic_sta_lta6_mean'] = classic_sta_lta(x, 100, 5000).mean()
    X_features.loc['classic_sta_lta7_mean'] = classic_sta_lta(x, 333, 666).mean()
    X_features.loc['classic_sta_lta8_mean'] = classic_sta_lta(x, 4000, 10000).mean()
    
#     no_of_std = 3
#     X_features.loc['Moving_average_700_mean'] = x.rolling(window=700).mean().mean(skipna=True)
#     X_features.loc['MA_700MA_std_mean'] = x.rolling(window=700).std().mean()
#     X_features.loc['MA_700MA_BB_highf_mean'] = (X_features.loc['Moving_average_700_mean'] + no_of_std * X_features.loc['MA_700MA_std_mean']).mean()
#     X_features.loc['MA_700MA_BB_low_mean'] = (X_features.loc['Moving_average_700_mean'] - no_of_std * X_features.loc['MA_700MA_std_mean']).mean()
#     X_features.loc['MA_400MA_std_mean'] = x.rolling(window=400).std().mean()
#     X_features.loc['MA_400MA_BB_high_mean'] = (X_features.loc['Moving_average_700_mean'] + no_of_std * X_features.loc['MA_400MA_std_mean']).mean()
#     X_features.loc['MA_400MA_BB_low_mean'] = (X_features.loc['Moving_average_700_mean'] - no_of_std * X_features.loc['MA_400MA_std_mean']).mean()
#     X_features.loc['MA_1000MA_std_mean'] = x.rolling(window=1000).std().mean()
#     X_features.drop('Moving_average_700_mean', inplace=True)
    
    X_features.loc['iqr'] = np.subtract(*np.percentile(x, [75, 25]))
    X_features.loc['ave10'] = stats.trim_mean(x, 0.1)

    X_features.loc[f'num_peaks_10'] = feature_calculators.number_peaks(x, 10)
    X_features.loc[f'c3_100'] = feature_calculators.c3(x, 100)
    X_features.loc[f'autocorrelation_5'] = feature_calculators.autocorrelation(x, 5)

    for windows in [10, 100, 1000]:
        x_roll_std = x.rolling(windows).std().dropna().values
        x_roll_mean = x.rolling(windows).mean().dropna().values
        
        X_features.loc['ave_roll_std_' + str(windows)] = x_roll_std.mean()
        X_features.loc['std_roll_std_' + str(windows)] = x_roll_std.std()
        X_features.loc['max_roll_std_' + str(windows)] = x_roll_std.max()
        X_features.loc['min_roll_std_' + str(windows)] = x_roll_std.min()
        X_features.loc['q01_roll_std_' + str(windows)] = np.quantile(x_roll_std, 0.01)
        X_features.loc['q05_roll_std_' + str(windows)] = np.quantile(x_roll_std, 0.05)
        X_features.loc['q95_roll_std_' + str(windows)] = np.quantile(x_roll_std, 0.95)
        X_features.loc['q99_roll_std_' + str(windows)] = np.quantile(x_roll_std, 0.99)
        X_features.loc['av_change_abs_roll_std_' + str(windows)] = np.mean(np.diff(x_roll_std))
        X_features.loc['av_change_rate_roll_std_' + str(windows)] = np.mean(np.nonzero((np.diff(x_roll_std) / x_roll_std[:-1]))[0])
        #X_features.loc['abs_max_roll_std_' + str(windows)] = np.abs(x_roll_std).max()
        
        X_features.loc['ave_roll_mean_' + str(windows)] = x_roll_mean.mean()
        X_features.loc['std_roll_mean_' + str(windows)] = x_roll_mean.std()
        X_features.loc['max_roll_mean_' + str(windows)] = x_roll_mean.max()
        X_features.loc['min_roll_mean_' + str(windows)] = x_roll_mean.min()
        X_features.loc['q01_roll_mean_' + str(windows)] = np.quantile(x_roll_mean, 0.01)
        X_features.loc['q05_roll_mean_' + str(windows)] = np.quantile(x_roll_mean, 0.05)
        X_features.loc['q95_roll_mean_' + str(windows)] = np.quantile(x_roll_mean, 0.95)
        X_features.loc['q99_roll_mean_' + str(windows)] = np.quantile(x_roll_mean, 0.99)
        X_features.loc['av_change_abs_roll_mean_' + str(windows)] = np.mean(np.diff(x_roll_mean))
        X_features.loc['av_change_rate_roll_mean_' + str(windows)] = np.mean(np.nonzero((np.diff(x_roll_mean) / x_roll_mean[:-1]))[0])
        X_features.loc['abs_max_roll_mean_' + str(windows)] = np.abs(x_roll_mean).max()
        
    return X_features

In [42]:
def make_dataset(slide=150_000, window=10_000, val_eq_nums=[1,15,16]):
    train = []
    val = []

    for eq_num in tqdm(range(17)):
        data = pd.read_csv(DataPath/'train_wave_{}.csv'.format(eq_num))
        acoustic_data = data['acoustic_data'].values
        time_data = data['time_to_failure'].values
        for n in tqdm(range((len(data)-rows)//slide+1)):
            x = acoustic_data[slide*n:slide*n+rows]
            y = time_data[slide*n+rows-1]
            
            x = high_pass_filter(x, low_cutoff=10000, sample_rate=sample_rate)
            x = denoise_signal(x, wavelet='haar', level=1)
            
            feature_series = []
            for m in range(rows//window):
                x_mini = x[window*m:window*(m+1)]
                feature_series.append(feature_engineering(x_mini))
            
            x = np.array(pd.concat(feature_series, axis=1).values)
    
            if eq_num not in val_eq_nums:
                train.append((np.array(x,'float32'),(np.array([y],'float32'))))
            else:
                val.append((np.array(x,'float32'),(np.array([y],'float32'))))
        
        if eq_num == 0:
            break

    if len(val_eq_nums) == 0:
        return train

    return train, val

In [38]:
def make_dataset_parallel(slide=150_000, window=10_000, val_eq_nums=[1,15,16]):
    train = []
    val = []

    for eq_num in tqdm(range(17)):
        data = pd.read_csv(DataPath/'train_wave_{}.csv'.format(eq_num))
        acoustic_data = data['acoustic_data'].values
        time_data = data['time_to_failure'].values

        def process(n):
            x = acoustic_data[slide*n:slide*n+rows]
            y = time_data[slide*n+rows-1]
            
            x = high_pass_filter(x, low_cutoff=10000, sample_rate=sample_rate)
            x = denoise_signal(x, wavelet='haar', level=1)
            
            feature_series = []
            for m in range(rows//window):
                x_mini = x[window*m:window*(m+1)]
                feature_series.append(feature_engineering(x_mini))
            
            x = np.array(pd.concat(feature_series, axis=1).values)
            
            x = np.array(x,'float32')
            y = np.array([y],'float32')
            
            return x, y              
        
        eq_data = Parallel(n_jobs=-1, verbose=5)([delayed(process)(n) for n in range((len(data)-rows)//slide+1)])
    
        if eq_num not in val_eq_nums:
            train = train + eq_data
        else:
            val = val + eq_data
        
        if eq_num == 2:
            break

    if len(val_eq_nums) == 0:
        return train

    return train, val

In [43]:
train, val =  make_dataset()

In [44]:
len(train), train[0][0].shape

(37, (105, 15))

In [45]:
train2, val2 =  make_dataset_parallel()

[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done   2 tasks      | elapsed:    4.5s
[Parallel(n_jobs=-1)]: Done  30 out of  37 | elapsed:   12.3s remaining:    2.9s
[Parallel(n_jobs=-1)]: Done  37 out of  37 | elapsed:   14.0s finished


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done   2 tasks      | elapsed:    3.0s
[Parallel(n_jobs=-1)]: Done  56 tasks      | elapsed:   20.1s
[Parallel(n_jobs=-1)]: Done 146 tasks      | elapsed:   51.8s
[Parallel(n_jobs=-1)]: Done 272 tasks      | elapsed:  1.5min
[Parallel(n_jobs=-1)]: Done 296 out of 296 | elapsed:  1.7min finished


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done   2 tasks      | elapsed:    3.1s
[Parallel(n_jobs=-1)]: Done  56 tasks      | elapsed:   19.0s
[Parallel(n_jobs=-1)]: Done 146 tasks      | elapsed:   50.0s
[Parallel(n_jobs=-1)]: Done 272 tasks      | elapsed:  1.5min
[Parallel(n_jobs=-1)]: Done 363 out of 363 | elapsed:  2.0min finished


In [47]:
len(train2), val2[0][0].shape

(400, (105, 15))